<a href="https://colab.research.google.com/github/christinaxliu/research/blob/main/Caltech-JPL-Intern/ExoplanetClassifiers/KNNClassifier/Exoplanet_KNN_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, normalize
from scipy import ndimage
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pickle
from google.colab import drive

In [27]:
warnings.filterwarnings('ignore')
pd.options.mode.copy_on_write = True
plt.style.use('fivethirtyeight')

In [28]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
exoplanets_filename = '/content/drive/My Drive/Colab Notebooks/ExoplanetClassifiers/KNNClassifier/TESSExoplanetCandidates_2024.06.23.csv'
exoplanets_data = pd.read_table(exoplanets_filename,
                                skiprows=54,
                                sep=',',
                                header=None,
                                index_col=None,
                                names = ['toi', 'tid', 'tfopwg_disp', 'ra', 'dec', 'st_pmra', 'st_pmraerr1', 'st_pmraerr2',
                                          'st_pmdec', 'st_pmdecerr1', 'st_pmdecerr2', 'pl_tranmid', 'pl_tranmiderr1', 'pl_tranmiderr2',
                                          'pl_orbper', 'pl_orbpererr1', 'pl_orbpererr2', 'pl_trandurh', 'pl_trandurherr1', 'pl_trandurherr2',
                                          'pl_trandep', 'pl_trandeperr1', 'pl_trandeperr2', 'pl_rade', 'pl_radeerr1', 'pl_radeerr2',
                                          'pl_insol', 'pl_insolerr1', 'pl_insolerr2', 'pl_eqt', 'pl_eqterr1', 'pl_eqterr2',
                                          'st_tmag', 'st_tmagerr1', 'st_tmagerr2', 'st_dist', 'st_disterr1', 'st_disterr2',
                                          'st_teff', 'st_tefferr1', 'st_tefferr2', 'st_logg', 'st_loggerr1', 'st_loggerr2',
                                          'st_rad', 'st_raderr1', 'st_raderr2'],
                                skipfooter=0,
                                engine='python')

In [30]:
# Sanity check the first several rows of data to ensure data is read correctly
exoplanets_data.head()

,toi,tid,tfopwg_disp,ra,dec,st_pmra,st_pmraerr1,st_pmraerr2,st_pmdec,st_pmdecerr1,...,st_disterr2,st_teff,st_tefferr1,st_tefferr2,st_logg,st_loggerr1,st_loggerr2,st_rad,st_raderr1,st_raderr2
0,1000.01,50365310,FP,112.357708,-12.695960,-5.964,0.085,-0.085,-0.076,0.072,...,-11.9515,10249.0,264.7,-264.7,4.19,0.07,-0.07,2.16986,0.072573,-0.072573
1,1001.01,88863718,PC,122.580465,-5.513852,-4.956,0.102,-0.102,-15.555,0.072,...,-5.9100,7070.0,126.4,-126.4,4.03,0.09,-0.09,2.01000,0.090000,-0.090000
2,1002.01,124709665,FP,104.726966,-10.580455,-1.462,0.206,-0.206,-2.249,0.206,...,-106.3330,8924.0,124.0,-124.0,NaN,NaN,NaN,5.73000,NaN,NaN
3,1003.01,106997505,FP,110.559945,-25.207017,-0.939,0.041,-0.041,1.640,0.055,...,-1899.5700,5388.5,567.0,-567.0,4.15,1.64,-1.64,NaN,NaN,NaN
4,1004.01,238597883,FP,122.178195,-48.802811,-4.496,0.069,-0.069,9.347,0.062,...,-4.6175,9219.0,171.1,-171.1,4.14,0.07,-0.07,2.15000,0.060000,-0.060000


In [31]:
# Sanity check the last several rows of data to ensure data is read correctly.
exoplanets_data.tail()

,toi,tid,tfopwg_disp,ra,dec,st_pmra,st_pmraerr1,st_pmraerr2,st_pmdec,st_pmdecerr1,...,st_disterr2,st_teff,st_tefferr1,st_tefferr2,st_logg,st_loggerr1,st_loggerr2,st_rad,st_raderr1,st_raderr2
7198,995.01,317951248,FP,110.811443,5.562850,2.061,0.405,-0.405,-7.082,0.336,...,-355.6340,4805.2,46.5,-46.5,NaN,NaN,NaN,NaN,NaN,NaN
7199,996.01,142918609,FP,119.349948,-19.516015,-3.900,0.848,-0.848,3.866,1.158,...,NaN,8007.0,NaN,NaN,NaN,NaN,NaN,2.050000,NaN,NaN
7200,997.01,341729521,FP,121.319521,-59.579798,-44.770,0.044,-0.044,-2.616,0.036,...,-0.2655,5786.0,132.9,-132.9,4.52,0.08,-0.08,0.926261,0.045789,-0.045789
7201,998.01,54390047,FP,118.319555,-14.218823,-1.706,0.069,-0.069,-1.714,0.061,...,-11.1405,8322.0,130.7,-130.7,4.01,0.07,-0.07,2.349860,0.091578,-0.091578
7202,999.01,341186896,FP,118.864086,-58.222060,-17.190,0.042,-0.042,12.272,0.039,...,-0.6635,6635.0,130.1,-130.1,4.35,0.09,-0.09,1.300000,0.050000,-0.050000


In [32]:
# Get the statistics about the data
exoplanets_data.describe()

,toi,tid,ra,dec,st_pmra,st_pmraerr1,st_pmraerr2,st_pmdec,st_pmdecerr1,st_pmdecerr2,...,st_disterr2,st_teff,st_tefferr1,st_tefferr2,st_logg,st_loggerr1,st_loggerr2,st_rad,st_raderr1,st_raderr2
count,7203.000000,7.203000e+03,7203.000000,7203.000000,7084.000000,7084.000000,7084.000000,7084.000000,7084.000000,7084.000000,...,6559.000000,7072.000000,6776.000000,6776.000000,6408.000000,5117.000000,5117.000000,6733.000000,5402.000000,5402.000000
mean,3514.036440,2.448209e+08,173.954915,-0.650568,-0.884084,0.227741,-0.227741,-9.083052,0.221717,-0.221717,...,-20.059656,5797.592605,210.114839,-210.114839,4.305612,0.180526,-0.180526,1.406487,0.075384,-0.075384
std,2013.842205,1.635179e+08,101.316743,47.868050,78.855921,0.635495,0.635495,68.206643,0.625434,0.625434,...,141.650242,1502.824027,567.587345,567.587345,0.305031,0.361619,0.361619,1.642425,0.085912,0.085912
min,101.010000,2.876000e+03,0.084680,-89.471513,-1624.050000,0.015000,-8.000000,-1230.620000,0.016000,-8.000000,...,-4020.600000,2808.000000,7.824290,-7000.000000,0.100000,0.000391,-2.010840,0.114827,0.003357,-1.723020
25%,1756.510000,1.303825e+08,93.891644,-44.950389,-11.083000,0.038000,-0.080000,-14.749000,0.038000,-0.071000,...,-11.875250,5203.925000,122.000000,-157.000000,4.130000,0.080000,-0.095776,0.890000,0.050000,-0.080000
50%,3494.010000,2.454624e+08,153.779587,0.444704,-1.787000,0.051000,-0.051000,-3.331500,0.049000,-0.049000,...,-4.154500,5801.000000,129.557500,-129.557500,4.330000,0.085884,-0.085884,1.233690,0.060000,-0.060000
75%,5262.510000,3.530925e+08,272.132619,42.672596,8.134750,0.080000,-0.038000,5.154000,0.071000,-0.038000,...,-0.988500,6301.475000,157.000000,-122.000000,4.500000,0.095776,-0.080000,1.660000,0.080000,-0.050000
max,7031.010000,2.041563e+09,359.933006,89.086923,2074.520000,8.000000,-0.015000,1048.840000,8.000000,-0.016000,...,-0.002835,50000.000000,7000.000000,-7.824290,5.960650,2.010840,-0.000391,102.030000,1.723020,-0.003357


In [33]:
# Cast numerical values to floats
exoplanets_data['ra'] = exoplanets_data['ra'].astype(float)
exoplanets_data['dec'] = exoplanets_data['dec'].astype(float)
exoplanets_data['st_pmra'] = exoplanets_data['st_pmra'].astype(float)
exoplanets_data['st_pmraerr1'] = exoplanets_data['st_pmraerr1'].astype(float)
exoplanets_data['st_pmraerr2'] = exoplanets_data['st_pmraerr2'].astype(float)
exoplanets_data['st_pmdec'] = exoplanets_data['st_pmdec'].astype(float)
exoplanets_data['st_pmdecerr1'] = exoplanets_data['st_pmdecerr1'].astype(float)
exoplanets_data['st_pmdecerr2'] = exoplanets_data['st_pmdecerr2'].astype(float)
exoplanets_data['pl_tranmid'] = exoplanets_data['pl_tranmid'].astype(float)
exoplanets_data['pl_tranmiderr1'] = exoplanets_data['pl_tranmiderr1'].astype(float)
exoplanets_data['pl_tranmiderr2'] = exoplanets_data['pl_tranmiderr2'].astype(float)
exoplanets_data['pl_orbper'] = exoplanets_data['pl_orbper'].astype(float)
exoplanets_data['pl_orbpererr1'] = exoplanets_data['pl_orbpererr1'].astype(float)
exoplanets_data['pl_orbpererr2'] = exoplanets_data['pl_orbpererr2'].astype(float)
exoplanets_data['pl_trandurh'] = exoplanets_data['pl_trandurh'].astype(float)
exoplanets_data['pl_trandurherr1'] = exoplanets_data['pl_trandurherr1'].astype(float)
exoplanets_data['pl_trandurherr2'] = exoplanets_data['pl_trandurherr2'].astype(float)
exoplanets_data['pl_trandep'] = exoplanets_data['pl_trandep'].astype(float)
exoplanets_data['pl_trandeperr1'] = exoplanets_data['pl_trandeperr1'].astype(float)
exoplanets_data['pl_trandeperr2'] = exoplanets_data['pl_trandeperr2'].astype(float)
exoplanets_data['pl_rade'] = exoplanets_data['pl_rade'].astype(float)
exoplanets_data['pl_radeerr1'] = exoplanets_data['pl_radeerr1'].astype(float)
exoplanets_data['pl_radeerr2'] = exoplanets_data['pl_radeerr2'].astype(float)
exoplanets_data['pl_insol'] = exoplanets_data['pl_insol'].astype(float)
exoplanets_data['pl_insolerr1'] = exoplanets_data['pl_insolerr1'].astype(float)
exoplanets_data['pl_insolerr2'] = exoplanets_data['pl_insolerr2'].astype(float)
exoplanets_data['pl_eqt'] = exoplanets_data['pl_eqt'].astype(float)
exoplanets_data['pl_eqterr1'] = exoplanets_data['pl_eqterr1'].astype(float)
exoplanets_data['pl_eqterr2'] = exoplanets_data['pl_eqterr2'].astype(float)
exoplanets_data['st_tmag'] = exoplanets_data['st_tmag'].astype(float)
exoplanets_data['st_tmagerr1'] = exoplanets_data['st_tmagerr1'].astype(float)
exoplanets_data['st_tmagerr2'] = exoplanets_data['st_tmagerr2'].astype(float)
exoplanets_data['st_dist'] = exoplanets_data['st_dist'].astype(float)
exoplanets_data['st_disterr1'] = exoplanets_data['st_disterr1'].astype(float)
exoplanets_data['st_disterr2'] = exoplanets_data['st_disterr2'].astype(float)
exoplanets_data['st_teff'] = exoplanets_data['st_teff'].astype(float)
exoplanets_data['st_tefferr1'] = exoplanets_data['st_tefferr1'].astype(float)
exoplanets_data['st_tefferr2'] = exoplanets_data['st_tefferr2'].astype(float)
exoplanets_data['st_logg'] = exoplanets_data['st_logg'].astype(float)
exoplanets_data['st_loggerr1'] = exoplanets_data['st_loggerr1'].astype(float)
exoplanets_data['st_loggerr2'] = exoplanets_data['st_loggerr2'].astype(float)
exoplanets_data['st_rad'] = exoplanets_data['st_rad'].astype(float)
exoplanets_data['st_raderr1'] = exoplanets_data['st_raderr1'].astype(float)
exoplanets_data['st_raderr2'] = exoplanets_data['st_raderr2'].astype(float)

In [34]:
# Change any fields with empty spaces to NaN values so we could clean them up later on
exoplanets_data = exoplanets_data.applymap(lambda x: np.nan if isinstance(x, str) and x.isspace() else x)

In [35]:
exoplanets_data['tfopwg_disp'].describe()

count     7201
unique       6
top         PC
freq      4657
Name: tfopwg_disp, dtype: object

In [36]:
exoplanets_data['tfopwg_disp'].value_counts()

tfopwg_disp
PC     4657
FP     1034
KP      537
CP      459
APC     422
FA       92
Name: count, dtype: int64

In [37]:
exoplanets_data = exoplanets_data.dropna(subset=['tfopwg_disp'])
exoplanets_data.describe()

,toi,tid,ra,dec,st_pmra,st_pmraerr1,st_pmraerr2,st_pmdec,st_pmdecerr1,st_pmdecerr2,...,st_disterr2,st_teff,st_tefferr1,st_tefferr2,st_logg,st_loggerr1,st_loggerr2,st_rad,st_raderr1,st_raderr2
count,7201.000000,7.201000e+03,7201.000000,7201.000000,7082.000000,7082.00000,7082.00000,7082.000000,7082.000000,7082.000000,...,6557.000000,7070.000000,6774.000000,6774.000000,6406.000000,5115.000000,5115.000000,6731.000000,5400.000000,5400.000000
mean,3514.043808,2.447937e+08,173.975150,-0.641553,-0.894011,0.22775,-0.22775,-9.073147,0.221746,-0.221746,...,-20.065663,5797.879335,210.144294,-210.144294,4.305494,0.180483,-0.180483,1.406639,0.075399,-0.075399
std,2013.411388,1.635242e+08,101.310846,47.869494,78.855069,0.63558,0.63558,68.212533,0.625519,0.625519,...,141.671429,1502.624980,567.668490,567.668490,0.304901,0.361660,0.361660,1.642614,0.085923,0.085923
min,101.010000,2.876000e+03,0.084680,-89.471513,-1624.050000,0.01500,-8.00000,-1230.620000,0.016000,-8.000000,...,-4020.600000,2808.000000,7.824290,-7000.000000,0.100000,0.000391,-2.010840,0.114827,0.003357,-1.723020
25%,1757.010000,1.303497e+08,93.921031,-44.938923,-11.075750,0.03800,-0.08000,-14.722500,0.038000,-0.071000,...,-11.876000,5204.325000,122.000000,-157.000000,4.130000,0.080000,-0.095659,0.890000,0.050000,-0.080000
50%,3494.010000,2.454624e+08,153.779587,0.458798,-1.787000,0.05100,-0.05100,-3.325500,0.049000,-0.049000,...,-4.161000,5801.000000,129.580500,-129.580500,4.330000,0.085750,-0.085750,1.233690,0.060000,-0.060000
75%,5262.010000,3.530387e+08,272.204520,42.677739,8.128250,0.08000,-0.03800,5.154000,0.071000,-0.038000,...,-0.988500,6301.225000,157.000000,-122.000000,4.500000,0.095659,-0.080000,1.660000,0.080000,-0.050000
max,7031.010000,2.041563e+09,359.933006,89.086923,2074.520000,8.00000,-0.01500,1048.840000,8.000000,-0.016000,...,-0.002835,50000.000000,7000.000000,-7.824290,5.960650,2.010840,-0.000391,102.030000,1.723020,-0.003357


In [38]:
# Label CP (confirmed planet) and KP (known planet) as postive samples and FP (false positive) as negative samples
exoplanets_data.loc[((exoplanets_data['tfopwg_disp'] == "CP") | (exoplanets_data['tfopwg_disp'] == "KP")), 'label'] = 1
exoplanets_data.loc[(exoplanets_data['tfopwg_disp'] == "FP"), 'label'] = 0

In [39]:
exoplanets_data['label'].value_counts()

label
0.0    1034
1.0     996
Name: count, dtype: int64

In [40]:
# Drop data fields that are not relevant to training or don't provide value to training
exoplanet_features_data = exoplanets_data.drop(['toi'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['tid'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['tfopwg_disp'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['st_pmraerr1'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['st_pmraerr2'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['st_pmdecerr1'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['st_pmdecerr2'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['pl_tranmiderr1'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['pl_tranmiderr2'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['pl_orbpererr1'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['pl_orbpererr2'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['pl_trandurherr1'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['pl_trandurherr2'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['pl_trandeperr1'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['pl_trandeperr2'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['pl_radeerr1'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['pl_radeerr2'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['pl_insolerr1'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['pl_insolerr2'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['pl_eqterr1'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['pl_eqterr2'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['st_tmagerr1'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['st_tmagerr2'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['st_disterr1'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['st_disterr2'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['st_tefferr1'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['st_tefferr2'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['st_loggerr1'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['st_loggerr2'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['st_raderr1'], axis = 1)
exoplanet_features_data = exoplanet_features_data.drop(['st_raderr2'], axis = 1)
exoplanet_features_data.to_csv('/content/drive/My Drive/Colab Notebooks/ExoplanetClassifiers/KNNClassifier/FeaturesData_DropInrelevantFields_2024.06.23.csv')
exoplanet_features_data

,ra,dec,st_pmra,st_pmdec,pl_tranmid,pl_orbper,pl_trandurh,pl_trandep,pl_rade,pl_insol,pl_eqt,st_tmag,st_dist,st_teff,st_logg,st_rad,label
0,112.357708,-12.695960,-5.964,-0.076,2.459230e+06,2.171348,2.017220,656.886099,5.818163,22601.948581,3127.204052,9.604000,485.735,10249.0,4.19,2.169860,0.0
1,122.580465,-5.513852,-4.956,-15.555,2.459988e+06,1.931646,3.166000,1286.000000,11.215400,44464.500000,4045.000000,9.423440,295.862,7070.0,4.03,2.010000,NaN
2,104.726966,-10.580455,-1.462,-2.249,2.459225e+06,1.867557,1.408000,1500.000000,23.752900,2860.610000,2037.000000,9.299501,943.109,8924.0,NaN,5.730000,0.0
3,110.559945,-25.207017,-0.939,1.640,2.458493e+06,2.743230,3.167000,383.410000,NaN,1177.360000,1631.000000,9.300300,7728.170,5388.5,4.15,NaN,0.0
4,122.178195,-48.802811,-4.496,9.347,2.459987e+06,3.573014,3.370000,755.000000,11.311300,54679.300000,4260.000000,9.135500,356.437,9219.0,4.14,2.150000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7198,110.811443,5.562850,2.061,-7.082,2.458494e+06,3.443800,2.572000,7260.750000,NaN,1413.670000,1708.000000,10.078100,1080.880,4805.2,NaN,NaN,0.0
7199,119.349948,-19.516015,-3.900,3.866,2.458495e+06,14.537800,6.826000,4040.000000,16.052300,NaN,NaN,9.792500,379.693,8007.0,NaN,2.050000,0.0
7200,121.319521,-59.579798,-44.770,-2.616,2.459230e+06,8.413486,3.556833,17479.605331,20.510696,127.916421,857.731431,9.772700,113.667,5786.0,4.52,0.926261,0.0
7201,118.319555,-14.218823,-1.706,-1.714,2.459230e+06,0.941436,1.360700,339.912662,4.529209,41562.587811,3641.626449,9.695200,482.278,8322.0,4.01,2.349860,0.0


In [41]:
# Drop off data rows without labels to get training data
training_data = exoplanet_features_data.loc[~np.isnan(exoplanet_features_data['label'])]
training_data['label'].value_counts()
training_data.describe()

,ra,dec,st_pmra,st_pmdec,pl_tranmid,pl_orbper,pl_trandurh,pl_trandep,pl_rade,pl_insol,pl_eqt,st_tmag,st_dist,st_teff,st_logg,st_rad,label
count,2030.000000,2030.000000,2008.000000,2008.000000,2.030000e+03,2007.000000,2030.000000,2030.000000,1921.000000,1972.000000,1946.000000,2030.000000,1993.000000,1997.000000,1846.000000,1921.000000,2030.000000
mean,176.529115,1.920753,1.838150,-14.920173,2.459321e+06,8.935517,3.007747,8624.570937,10.743431,4144.881653,1461.486768,10.933019,417.322841,5852.984552,4.308429,1.531377,0.490640
std,106.558725,46.260083,126.013408,96.837248,5.867907e+02,51.123985,1.698272,22648.929982,13.006856,16019.719596,845.372887,1.706347,630.093878,1598.774543,0.322753,2.830962,0.500036
min,0.185606,-89.471513,-1624.050000,-1230.620000,2.458325e+06,0.247668,0.199000,100.000000,0.757588,0.018728,103.000000,4.627800,6.531270,2828.000000,2.360000,0.114827,0.000000
25%,90.549560,-39.586708,-13.209000,-18.540750,2.458766e+06,1.901212,1.881640,1134.172500,4.026926,120.412250,895.695726,9.884435,140.225000,5134.000000,4.110000,0.864173,0.000000
50%,150.978382,2.229966,-1.670000,-3.927500,2.459323e+06,3.522475,2.675500,3729.817355,10.218500,575.036500,1312.498240,10.749600,280.355000,5817.300000,4.340000,1.255050,0.000000
75%,287.596915,44.248476,11.763750,5.581500,2.459824e+06,6.879605,3.675418,10220.532790,14.222600,1978.815840,1797.750000,12.098875,494.985000,6327.160000,4.510000,1.720000,1.000000
max,359.900874,87.868530,2074.520000,1048.840000,2.460406e+06,1825.048364,24.052709,504062.379922,297.111726,280833.000000,6413.000000,18.332400,8341.800000,31000.000000,5.599980,102.030000,1.000000


In [42]:
# Apply simple imputation by filling in missing values with mean
training_data = training_data.fillna(training_data.mean())
training_data = training_data.dropna()
training_data.to_csv('/content/drive/My Drive/Colab Notebooks/ExoplanetClassifiers/KNNClassifier/TrainingData_Final_2024.06.23.csv')
training_data

,ra,dec,st_pmra,st_pmdec,pl_tranmid,pl_orbper,pl_trandurh,pl_trandep,pl_rade,pl_insol,pl_eqt,st_tmag,st_dist,st_teff,st_logg,st_rad,label
0,112.357708,-12.695960,-5.964,-0.076,2.459230e+06,2.171348,2.017220,656.886099,5.818163,22601.948581,3127.204052,9.604000,485.735,10249.0,4.190000,2.169860,0.0
2,104.726966,-10.580455,-1.462,-2.249,2.459225e+06,1.867557,1.408000,1500.000000,23.752900,2860.610000,2037.000000,9.299501,943.109,8924.0,4.308429,5.730000,0.0
3,110.559945,-25.207017,-0.939,1.640,2.458493e+06,2.743230,3.167000,383.410000,10.743431,1177.360000,1631.000000,9.300300,7728.170,5388.5,4.150000,1.531377,0.0
4,122.178195,-48.802811,-4.496,9.347,2.459987e+06,3.573014,3.370000,755.000000,11.311300,54679.300000,4260.000000,9.135500,356.437,9219.0,4.140000,2.150000,0.0
5,120.704811,-11.101521,-26.932,-2.901,2.458493e+06,4.550720,2.599000,3620.000000,6.544490,254.050000,1112.000000,9.130900,100.711,5613.0,4.308429,1.090000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7198,110.811443,5.562850,2.061,-7.082,2.458494e+06,3.443800,2.572000,7260.750000,10.743431,1413.670000,1708.000000,10.078100,1080.880,4805.2,4.308429,1.531377,0.0
7199,119.349948,-19.516015,-3.900,3.866,2.458495e+06,14.537800,6.826000,4040.000000,16.052300,4144.881653,1461.486768,9.792500,379.693,8007.0,4.308429,2.050000,0.0
7200,121.319521,-59.579798,-44.770,-2.616,2.459230e+06,8.413486,3.556833,17479.605331,20.510696,127.916421,857.731431,9.772700,113.667,5786.0,4.520000,0.926261,0.0
7201,118.319555,-14.218823,-1.706,-1.714,2.459230e+06,0.941436,1.360700,339.912662,4.529209,41562.587811,3641.626449,9.695200,482.278,8322.0,4.010000,2.349860,0.0


In [43]:
features = training_data.drop(['label'], axis = 1)
results = training_data.label

In [44]:
# Split data with 70% for training and 30% for testing
features_train, features_test, results_train, results_test = train_test_split(features, results, test_size = 0.3, random_state = 0)

In [45]:
# Standadize the scales of features
sc = StandardScaler()
features_train_sc = sc.fit_transform(features_train)
features_test_sc = sc.transform(features_test)

In [20]:
#features_train = normalized = normalize(features_train)
#features_test = normalize(features_test)

In [21]:
#features_train = filtered = ndimage.filters.gaussian_filter(features_train, sigma=10)
#features_test = ndimage.filters.gaussian_filter(features_test, sigma=10)

In [22]:
#std_scaler = StandardScaler()
#features_train_sc = scaled = std_scaler.fit_transform(features_train)
#features_test_sc = std_scaler.fit_transform(features_test)

In [46]:
# Train a KNN classifier with the training data
knn_classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
knn_classifier.fit(features_train_sc, results_train)

KNeighborsClassifier()

In [47]:
# Call trained KNN classifier to get predictions for the testing data
results_pred = knn_classifier.predict(features_test_sc)

In [48]:
# Calculate accuracy, precision, recall, and F-1 scores for the KNN classifier
print("KNN Classifier Accuracy: ", accuracy_score(results_test, results_pred))
print()
print("KNN Classifier Classification Report :\n",(classification_report(results_test,results_pred)))

KNN Classifier Accuracy:  0.8095238095238095

KNN Classifier Classification Report :
               precision    recall  f1-score   support

         0.0       0.90      0.71      0.79       309
         1.0       0.75      0.92      0.83       300

    accuracy                           0.81       609
   macro avg       0.82      0.81      0.81       609
weighted avg       0.83      0.81      0.81       609



In [49]:
# Save the KNN classifier model onto disk
knnPickle = open('/content/drive/My Drive/Colab Notebooks/ExoplanetClassifiers/KNNClassifier/exoplanet_knn_classifier_v1', 'wb')
pickle.dump(knn_classifier, knnPickle)
knnPickle.close()

In [50]:
# Load the KNN classifier model from disk and test with test data
loaded_knn_classifier = pickle.load(open('/content/drive/My Drive/Colab Notebooks/ExoplanetClassifiers/KNNClassifier/exoplanet_knn_classifier_v1', 'rb'))
test_pred_results = loaded_knn_classifier.predict(features_test_sc)
test_pred_results

array([0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
       0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0.,
       0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1.,
       1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1.,
       1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1.,
       1., 0., 1., 1., 0.

In [51]:
# The data rows with NaN labels are exoplanet candidates
candidate_exoplanets = exoplanet_features_data.loc[np.isnan(exoplanet_features_data['label'])]
candidate_exoplanets = candidate_exoplanets.drop(['label'], axis = 1)
candidate_exoplanets.describe()

,ra,dec,st_pmra,st_pmdec,pl_tranmid,pl_orbper,pl_trandurh,pl_trandep,pl_rade,pl_insol,pl_eqt,st_tmag,st_dist,st_teff,st_logg,st_rad
count,5171.000000,5171.000000,5074.000000,5074.000000,5.171000e+03,5092.000000,5171.000000,5171.000000,4810.000000,5079.000000,4980.000000,5171.000000,5001.000000,5073.000000,4560.000000,4810.000000
mean,172.972530,-1.647448,-1.975244,-6.759228,2.459521e+06,22.132351,3.062630,7916.216182,10.150587,1413.704845,1212.393324,11.751034,497.943562,5776.187020,4.304306,1.356821
std,99.167462,48.454063,48.918294,52.591379,5.032803e+02,114.176194,1.894036,11359.058667,6.171677,6553.308426,590.686114,1.550294,541.577126,1462.637987,0.297398,0.753362
min,0.084680,-88.568495,-840.114000,-1098.040000,2.457926e+06,0.152076,0.101000,23.453970,0.552507,0.000342,37.000000,5.289060,10.454000,2808.000000,0.100000,0.125969
25%,95.115910,-47.023823,-10.439500,-13.615000,2.459232e+06,2.689045,1.824944,1557.270908,4.563078,75.644617,799.306351,10.684900,191.145000,5225.910000,4.130000,0.900000
50%,154.302943,-0.803340,-1.810000,-3.116500,2.459586e+06,4.346767,2.748000,5042.000000,10.619100,310.868000,1145.500000,12.122000,405.634000,5792.800000,4.330000,1.230000
75%,265.412802,42.205059,7.043250,5.025000,2.459910e+06,8.252952,3.811241,10334.109640,14.021275,978.904500,1529.000000,12.945450,688.605000,6287.200000,4.500000,1.630000
max,359.933006,89.086923,531.463000,795.850000,2.460421e+06,1837.889731,22.902650,225793.106096,140.194000,238125.935921,5634.059440,16.177900,14728.300000,50000.000000,5.960650,17.340000


In [52]:
candidate_exoplanets = candidate_exoplanets.fillna(candidate_exoplanets.mean())
candidate_exoplanets = candidate_exoplanets.dropna()
candidate_exoplanets.describe()

,ra,dec,st_pmra,st_pmdec,pl_tranmid,pl_orbper,pl_trandurh,pl_trandep,pl_rade,pl_insol,pl_eqt,st_tmag,st_dist,st_teff,st_logg,st_rad
count,5171.000000,5171.000000,5171.000000,5171.000000,5.171000e+03,5171.000000,5171.000000,5171.000000,5171.000000,5171.000000,5171.000000,5171.000000,5171.000000,5171.000000,5171.000000,5171.000000
mean,172.972530,-1.647448,-1.975244,-6.759228,2.459521e+06,22.132351,3.062630,7916.216182,10.150587,1413.704845,1212.393324,11.751034,497.943562,5776.187020,4.304306,1.356821
std,99.167462,48.454063,48.457217,52.095680,5.032803e+02,113.300503,1.894036,11359.058667,5.952307,6494.738729,579.672307,1.550294,532.598629,1448.709138,0.279272,0.726584
min,0.084680,-88.568495,-840.114000,-1098.040000,2.457926e+06,0.152076,0.101000,23.453970,0.552507,0.000342,37.000000,5.289060,10.454000,2808.000000,0.100000,0.125969
25%,95.115910,-47.023823,-10.164500,-13.199000,2.459232e+06,2.731434,1.824944,1557.270908,5.100595,78.148314,809.161269,10.684900,197.187000,5244.000000,4.160000,0.930000
50%,154.302943,-0.803340,-1.975244,-3.401000,2.459586e+06,4.416399,2.748000,5042.000000,10.150587,324.912095,1169.000000,12.122000,425.694000,5780.000000,4.304306,1.290000
75%,265.412802,42.205059,6.730000,4.818500,2.459910e+06,8.609472,3.811241,10334.109640,13.720400,1052.639871,1513.000000,12.945450,677.752000,6277.500000,4.470000,1.590000
max,359.933006,89.086923,531.463000,795.850000,2.460421e+06,1837.889731,22.902650,225793.106096,140.194000,238125.935921,5634.059440,16.177900,14728.300000,50000.000000,5.960650,17.340000


In [53]:
candidate_exoplanets_sc = sc.transform(candidate_exoplanets)

In [54]:
candidate_pred_results = loaded_knn_classifier.predict(candidate_exoplanets_sc)
candidate_pred_results

array([0., 1., 0., ..., 0., 0., 0.])